In [4]:
!pip install moviepy==1.0.3
!pip install -U openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 17.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s

In [2]:
import whisper
from moviepy.editor import AudioFileClip
import os
import torch

# Define paths
base_path = "/content/sample_data"
audio_path = os.path.join(base_path, "Ignite.mp3")
audio_partition_folder = os.path.join(base_path, "audio_files")
subtitle_path = os.path.join(base_path, "subtitles", "subtitles.txt")

# Ensure directories exist
os.makedirs(audio_partition_folder, exist_ok=True)
os.makedirs(os.path.dirname(subtitle_path), exist_ok=True)

# Load audio and get duration
audio_clip = AudioFileClip(audio_path)
n = round(audio_clip.duration)
audio_clip.close()

# Partitioning parameters
start = 0
index = 60  # 1-minute segments
counter = 0

print("Partitioning the audio clip...")

# Partition the audio into 1-minute segments
while start < n:
    end_time = min(start + index, n)

    # Extract and save subclip
    temp_saving_location = os.path.join(audio_partition_folder, f'temp_{counter}.mp3')
    with AudioFileClip(audio_path) as audio_clip:
        temp = audio_clip.subclip(start, end_time)
        temp.write_audiofile(filename=temp_saving_location, verbose=False, logger=None)
        temp.close()

    start = end_time  # Move to next segment
    counter += 1

print("Partitioning completed.")

# Load Whisper model once
print("Loading Whisper model...")
model = whisper.load_model("large", device='cuda' if torch.cuda.is_available() else 'cpu')

# Transcribing each segment
print("Transcribing audio segments...")
final_list_of_text = []
id_counter = 0
start_time = 0

for index in range(counter):
    path_to_saved_file = os.path.join(audio_partition_folder, f'temp_{index}.mp3')

    # Ensure the file exists
    if not os.path.exists(path_to_saved_file):
        continue

    with AudioFileClip(path_to_saved_file) as audio_clip:
        duration = audio_clip.duration

    # Transcribe using Whisper
    out = model.transcribe(path_to_saved_file)
    list_of_text = out['segments']

    for line in list_of_text:
        line['start'] += start_time
        line['end'] += start_time
        line['id'] = id_counter
        id_counter += 1
        final_list_of_text.append(line)

    start_time += duration  # Update timestamp

    # Delete temporary file to free memory
    os.remove(path_to_saved_file)

# Save subtitles
with open(subtitle_path, 'w') as fp:
    for line in final_list_of_text:
        fp.write("{}\n".format(line.get("text")))

print("Successfully completed execution.")

Partitioning the audio clip...
Partitioning completed.
Loading Whisper model...
Transcribing audio segments...


  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



Successfully completed execution.
